# Using Semantic Kernel with MCP Servers

In this notebook, we'll connect a simple SK Agent to an MCP Server.

## 📋 Prerequisites & Configuration

### 🔧 **Environment Setup**
1. **Copy the environment file**:
   ```bash
   cp .env.example .env
   ```

2. **Edit `.env` file** with your actual credentials:
   - Azure OpenAI endpoint, API key, and deployment name
   - OR regular OpenAI API key
   
   The notebook will automatically load configuration from your `.env` file. 

3. **Start the MCP server** (see instructions below)

### 🗄️ **Database Setup**
The MCP server uses a SQLite database (`contoso.db`) that should be in the same directory.

Let's start our MCP server.

1. Open a new terminal and run:

    using uv:

    ```bash
    cd 03-semantic-kernel-mcp\
    uv run --prerelease=allow mcp_server.py
    ```

    using pip:
    ```bash
    pip install fastmcp
    cd .\03-semantic-kernel-mcp\
    python .\mcp_server.py
    ```

The server should come up like this:

```
INFO:     Started server process [49488]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
```

Great, ready to go, let's connect SK to it:


In [2]:
import os
from dotenv import load_dotenv
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPSsePlugin

# Load environment variables from .env file
load_dotenv()

print("🔧 Setting up MCP plugin...")

# Set up the SSE plugin for the MCP service.
contoso_mcp_plugin = MCPSsePlugin(
    name="ContosoMCP",
    description="Contoso MCP Plugin",
    url=os.getenv("MCP_SERVER_URL", "http://localhost:8000/sse"),
    headers={"Content-Type": "application/json"},
    timeout=30,
)

# Connect to MCP server
await contoso_mcp_plugin.connect()
print("✅ MCP plugin connected successfully!")

print("\n🤖 Setting up Semantic Kernel agent...")

# Configure Azure OpenAI service
service = AzureChatCompletion(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")
)

# Create the agent with MCP plugin
agent = ChatCompletionAgent(
    service=service,
    name="ChatBot",
    instructions="You are a helpful assistant. You can use multiple tools to find information "
    "and answer questions. Review the tools available under the ContosoMCP plugin "
    "and use them as needed. You can also ask clarifying questions if the user is not clear.",
    plugins=[contoso_mcp_plugin],
)

print("✅ Setup complete! Agent ready to help with customer inquiries.")

# Same as prior in our workshop 
thread: ChatHistoryAgentThread = None
user_messages = [
    "I noticed my last invoice was higher than usual—can you help me understand why and what can be done about it?",
    "My customer id is 42",
    ]

for user_message in user_messages:
    print("*** User:", user_message)
    response = await agent.get_response(messages=user_message, thread=thread)
    thread = response.thread
    print("*** Agent:", response.content)



🔧 Setting up MCP plugin...
✅ MCP plugin connected successfully!

🤖 Setting up Semantic Kernel agent...
✅ Setup complete! Agent ready to help with customer inquiries.
*** User: I noticed my last invoice was higher than usual—can you help me understand why and what can be done about it?
*** Agent: I can definitely help you with that! To get started, could you please provide your customer ID or the subscription ID associated with your account? This will help me look up your invoices and investigate the cause of the higher charge.
*** User: My customer id is 42
*** Agent: I reviewed your recent invoices for your "Contoso International Roaming" subscription, and here’s what I found:

- Your last invoice on 2025-04-26 was $129.71, which is indeed higher than your regular monthly fee of $20.00.
- Previous months' invoices varied, with some also being above your standard rate (e.g., $126.70, $123.67, $125.23).

This pattern suggests there may have been additional usage or extra charges applied

In [3]:
# Test the agent with customer inquiry
print("🗣️  Starting customer conversation...\n")

# Same as prior in our workshop 
thread: ChatHistoryAgentThread = None
user_messages = [
    "I noticed my last invoice was higher than usual—can you help me understand why and what can be done about it?",
    "My customer id is 42",
]

for user_message in user_messages:
    print(f"👤 User: {user_message}")
    
    response = await agent.get_response(messages=user_message, thread=thread)
    thread = response.thread
    print(f"🤖 Agent: {response.content}\n")
    print("-" * 80 + "\n")

print("✅ Conversation complete!")

🗣️  Starting customer conversation...

👤 User: I noticed my last invoice was higher than usual—can you help me understand why and what can be done about it?
🤖 Agent: I’d be happy to help you with your invoice concern! To assist you effectively, could you please provide your customer ID or any details related to your account or subscription? This will help me pull up your information and review your recent invoice for any discrepancies or reasons for the increase.

--------------------------------------------------------------------------------

👤 User: My customer id is 42
🤖 Agent: I reviewed your account and recent invoices. Here are my findings:

- Your total amount currently due is $490.23.
- Your most recent invoice (ID: 468) for $70.57 has not been paid because the most recent payment attempt (on 2025-02-24 using Apple Pay) failed.
- There are other outstanding invoices as well, which could explain the higher-than-usual balance.

Here’s what we can do next:
1. I can provide a brea